In [5]:
# Программа для разделение одного датасета на папки train и valid с созданием новых файлов анотаций изображений с помощью стратификации с эмбедингом
# Папка test была создана аналогичным образом отдельно, из файлов папки test изначального датасета

import os
import json
import random
import shutil
from pathlib import Path
from collections import Counter, defaultdict
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.io import read_image
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

# Создание структуру папок
data_dir = Path("data")
data_dir.mkdir(exist_ok=True)
(train_dir := data_dir / "train").mkdir(exist_ok=True)
(valid_dir := data_dir / "valid").mkdir(exist_ok=True)
(train_images_dir := train_dir / "images").mkdir(exist_ok=True)
(valid_images_dir := valid_dir / "images").mkdir(exist_ok=True)

Используемое устройство: cpu


In [6]:
# Датасет
class Dataset1(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        
        # Загружаем аннотации
        anno_path = self.root_dir / "_annotations.coco.json"
        with open(anno_path, 'r') as f:
            self.coco_data = json.load(f)
        
        self.image_id_to_info = {img['id']: img for img in self.coco_data['images']}
        self.category_id_to_name = {cat['id']: cat['name'] for cat in self.coco_data['categories']}
        
        self.image_id_to_annotations = defaultdict(list)
        for ann in self.coco_data['annotations']:
            self.image_id_to_annotations[ann['image_id']].append(ann)
        
        self.image_ids = list(self.image_id_to_info.keys())
        
        # Получаем пути для изображений
        self.image_paths = []
        for img_id in self.image_ids:
            img_info = self.image_id_to_info[img_id]
            self.image_paths.append(self.root_dir / img_info['file_name'])
    
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image_id = self.image_ids[idx]
        
        try:
            image = read_image(str(img_path)).float() / 255.0
        except:
            from PIL import Image
            img = Image.open(img_path).convert('RGB')
            image = transforms.ToTensor()(img)
        
        annotations = self.image_id_to_annotations[image_id]
        
        boxes = []
        labels = []
        
        for ann in annotations:
            # [x_min, y_min, width, height]
            x, y, w, h = ann['bbox']
            boxes.append([x, y, x + w, y + h])
            labels.append(ann['category_id'])
        
        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)
        
        if self.transform:
            image = self.transform(image)
        
        return image, {
            'image_id': image_id,
            'boxes': boxes,
            'labels': labels,
            'annotations': annotations}
    
    def get_image_info(self, idx):
        image_id = self.image_ids[idx]
        return self.image_id_to_info[image_id]
    
    def get_all_image_infos(self):
        return [self.image_id_to_info[img_id] for img_id in self.image_ids]


stratified_dir = Path("stratified_15percent")
dataset = Dataset1(stratified_dir)

print(f"   Изображений: {len(dataset)}")
print(f"   Категорий: {len(dataset.coco_data['categories'])}")
print(f"   Аннотаций: {len(dataset.coco_data['annotations'])}")

   Изображений: 1676
   Категорий: 3
   Аннотаций: 96248


In [7]:
# Извлечение эмбедингов
def extract_embeddings_from_dataset(dataset, batch_size=8):
    print("🧠 Извлекаем эмбединги из изображений...")
    
    # Используем предобученный ResNet18
    resnet = models.resnet18(pretrained=True)
    resnet = resnet.to(device)
    resnet.eval()
    
    feature_extractor = nn.Sequential(*list(resnet.children())[:-1])
    feature_extractor = feature_extractor.to(device)
    
    def extract_batch_embeddings(images):
        with torch.no_grad():
            if images.dim() == 3:
                images = images.unsqueeze(0)
            
            normalize = transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            )
            images = normalize(images)
            
            x = feature_extractor(images)  # [B, 512, 1, 1]
            x = x.view(x.size(0), -1)      # [B, 512]
        return x
    
    all_embeddings = []
    all_indices = []
    
    print(f"Извлекаем эмбединги для {len(dataset)} изображений...")
    
    # Извлекаем эмбединги батчами
    for i in tqdm(range(0, len(dataset), batch_size), desc="Извлечение эмбедингов"):
        batch_indices = list(range(i, min(i + batch_size, len(dataset))))
        batch_images = []
        
        for idx in batch_indices:
            image, _ = dataset[idx]
            batch_images.append(image)
        
        if not batch_images:
            continue
            
        try:
            batch_tensor = torch.stack(batch_images).to(device)
            
            # Извлекаем эмбединги
            embeddings = extract_batch_embeddings(batch_tensor)
            all_embeddings.append(embeddings.cpu())
            all_indices.extend(batch_indices)
        except Exception as e:
            continue
    
    all_embeddings = torch.cat(all_embeddings, dim=0).numpy()
    
    print(f"Извлечено эмбедингов: {all_embeddings.shape}")
    print(f"Размерность эмбединга: {all_embeddings.shape[1]}")
    print(f"Количество изображений: {len(all_indices)}")
    
    return all_embeddings, all_indices

embeddings, indices = extract_embeddings_from_dataset(dataset, batch_size=8)

🧠 Извлекаем эмбединги из изображений...
Извлекаем эмбединги для 1676 изображений...


Извлечение эмбедингов: 100%|██████████| 210/210 [12:49<00:00,  3.67s/it]

Извлечено эмбедингов: (1676, 512)
Размерность эмбединга: 512
Количество изображений: 1676


In [14]:
# K-means кластеризация
def cluster_embeddings(embeddings):
    
    n_clusters = min(10, len(embeddings) // 20)
    n_clusters = max(2, n_clusters)
    
    print(f"Используем {n_clusters} кластеров")
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(embeddings)
    
    cluster_counts = Counter(cluster_labels)
    print(f"Распределение по кластерам:")
    for cluster_id in range(n_clusters):
        count = cluster_counts.get(cluster_id, 0)
        percentage = count / len(cluster_labels) * 100
        print(f"Кластер {cluster_id}: {count} образцов")
    
    return cluster_labels, n_clusters

cluster_labels, n_clusters = cluster_embeddings(embeddings)

Используем 10 кластеров
Распределение по кластерам:
Кластер 0: 190 образцов
Кластер 1: 88 образцов
Кластер 2: 193 образцов
Кластер 3: 190 образцов
Кластер 4: 228 образцов
Кластер 5: 142 образцов
Кластер 6: 185 образцов
Кластер 7: 287 образцов
Кластер 8: 93 образцов
Кластер 9: 80 образцов


In [15]:
# Разделение на train/valid
def stratified_split_by_clusters(indices, cluster_labels, test_size=0.1, random_state=42):
    
    sss = StratifiedShuffleSplit(
        n_splits=1, 
        test_size=test_size,
        random_state=random_state
    )
    
    train_indices = []
    valid_indices = []
    
    for train_idx, valid_idx in sss.split(indices, cluster_labels):
        train_indices = [indices[i] for i in train_idx]
        valid_indices = [indices[i] for i in valid_idx]
    
    print()
    print(f"Train: {len(train_indices)} изображений")
    print(f"Valid: {len(valid_indices)} изображений")
    
    # Анализируем распределение по кластерам
    print(f"\nРаспределение по кластерам:")
    
    idx_to_cluster = {idx: cluster for idx, cluster in zip(indices, cluster_labels)}
    
    # Train распределение
    train_clusters = [idx_to_cluster[idx] for idx in train_indices]
    train_cluster_counts = Counter(train_clusters)
    
    print(f"Train:")
    for cluster_id in range(n_clusters):
        count = train_cluster_counts.get(cluster_id, 0)
        percentage = count / len(train_indices) * 100 if train_indices else 0
        print(f"Кластер {cluster_id}: {count} ({percentage:.1f}%)")
    
    # Valid распределение
    valid_clusters = [idx_to_cluster[idx] for idx in valid_indices]
    valid_cluster_counts = Counter(valid_clusters)
    
    print()
    print(f"Valid:")
    for cluster_id in range(n_clusters):
        count = valid_cluster_counts.get(cluster_id, 0)
        percentage = count / len(valid_indices) * 100 if valid_indices else 0
        print(f"Кластер {cluster_id}: {count} ({percentage:.1f}%)")
    
    return train_indices, valid_indices

# Разделяем на train и valid
train_indices, valid_indices = stratified_split_by_clusters(
    indices=indices,
    cluster_labels=cluster_labels,
    test_size=0.1,  # 10% для valid
    random_state=42
)


Train: 1508 изображений
Valid: 168 изображений

Распределение по кластерам:
Train:
Кластер 0: 171 (11.3%)
Кластер 1: 79 (5.2%)
Кластер 2: 174 (11.5%)
Кластер 3: 171 (11.3%)
Кластер 4: 205 (13.6%)
Кластер 5: 128 (8.5%)
Кластер 6: 166 (11.0%)
Кластер 7: 258 (17.1%)
Кластер 8: 84 (5.6%)
Кластер 9: 72 (4.8%)

Valid:
Кластер 0: 19 (11.3%)
Кластер 1: 9 (5.4%)
Кластер 2: 19 (11.3%)
Кластер 3: 19 (11.3%)
Кластер 4: 23 (13.7%)
Кластер 5: 14 (8.3%)
Кластер 6: 19 (11.3%)
Кластер 7: 29 (17.3%)
Кластер 8: 9 (5.4%)
Кластер 9: 8 (4.8%)


In [13]:
# Сохранение в папки
def save_dataset(dataset, indices, output_dir, split_name):

    # Создаем папку
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Подготавливаем структуру COCO JSON
    coco_output = {
        "info": { },
        "licenses": [],
        "categories": dataset.coco_data["categories"],
        "images": [],
        "annotations": [] }
    
    new_image_id = 1
    new_annotation_id = 1
    old_to_new_image_id = {}
    
    for idx in tqdm(indices):
        try:
            img_info = dataset.get_image_info(idx)
            original_image_id = img_info['id']
            
            original_image_path = dataset.image_paths[idx]
            
            new_image_filename = f"{split_name}_{new_image_id:06d}{original_image_path.suffix}"
            new_image_path = output_dir / new_image_filename
            
            shutil.copy2(original_image_path, new_image_path)
            
            new_image_info = {
                "id": new_image_id,
                "file_name": new_image_filename,
                "width": img_info['width'],
                "height": img_info['height'],
                "original_id": original_image_id,
                "original_filename": img_info['file_name']}
            coco_output["images"].append(new_image_info)
            
            old_to_new_image_id[original_image_id] = new_image_id
            
            # Получаем аннотации изображения
            annotations = dataset.image_id_to_annotations.get(original_image_id, [])
            
            for ann in annotations:
                new_annotation = {
                    "id": new_annotation_id,
                    "image_id": new_image_id,
                    "category_id": ann['category_id'],
                    "bbox": ann['bbox'],
                    "area": ann['area'],
                    "segmentation": ann.get('segmentation', []),
                    "iscrowd": ann.get('iscrowd', 0),
                    "original_id": ann['id'] }
                coco_output["annotations"].append(new_annotation)
                new_annotation_id += 1
            
            new_image_id += 1
            
        except Exception as e:
            print(f"{idx}: {e}")
            continue
    
    annotations_path = output_dir / "_annotations.coco.json"
    with open(annotations_path, 'w') as f:
        json.dump(coco_output, f, indent=2)
    
    print(f"Изображений: {len(coco_output['images'])}")
    print(f"Аннотаций: {len(coco_output['annotations'])}")
    
    return coco_output


# Сохраняем train
print("Сохранение train")
train_coco = save_dataset(
    dataset=dataset,
    indices=train_indices,
    output_dir=train_dir,
    split_name="train")

# Сохраняем valid
print("Сохранение val")
valid_coco = save_dataset(
    dataset=dataset,
    indices=valid_indices,
    output_dir=valid_dir,
    split_name="valid")

Сохранение train


100%|██████████| 1508/1508 [00:13<00:00, 115.02it/s]


Изображений: 1508
Аннотаций: 86508
Сохранение val


100%|██████████| 168/168 [00:02<00:00, 62.89it/s]


Изображений: 168
Аннотаций: 9740
